# Reto 3 (Científico de datos)
Julián David Montenegro (jdmontenegron@gmail.com)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Obtención de datos

En primer lugar, se obtiene el dataset.

In [2]:
set_datos = pd.read_csv("set_datos.csv", sep=";")
set_datos

,id_credito,numero_radicacion,numero_radicacion_credito_antiguo,fecha_originacion,valor_esperado,capital,interes,seguros,valor_recibido,porcentaje_incorporacion,...,altura_mora_calculada,saldo_vencido,saldo_capital,capital_mora,intereses_corrientes_mora,seguros_mora,intereses_mora,gastos_cobranza,tasa_mensual_credito,credito_nuevo
0,1693,1,NaN,2013-08-15,124094.0,56041.0,64633.0,3420,124094,100.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.1535,NO
1,1693,1,NaN,2013-08-15,124094.0,57248.0,63426.0,3420,124094,100.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.1535,NO
2,1693,1,NaN,2013-08-15,124094.0,58481.0,62193.0,3420,124094,100.0,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.1535,NO
3,1693,1,NaN,2013-08-15,124094.0,59740.0,60934.0,3420,124094,100.0,...,0,0.0,2769804.0,0.0,0.0,0.0,0.0,0.0,2.1535,NO
4,1693,1,NaN,2013-08-15,124094.0,61027.0,59647.0,3420,124094,100.0,...,0,0.0,2708777.0,0.0,0.0,0.0,0.0,0.0,2.1535,NO
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282762,66562,36684,NaN,2019-05-07,364347.0,31259.0,267881.0,65207,364347,100.0,...,0,0.0,14067741.0,0.0,0.0,0.0,0.0,0.0,1.9000,NO
282763,66563,36729,NaN,2019-05-20,182174.0,11971.0,139956.0,30247,182174,100.0,...,0,0.0,6528029.0,0.0,0.0,0.0,0.0,0.0,2.1400,NO
282764,66694,36826,NaN,2019-05-16,347757.0,26703.0,260755.0,60299,347757,100.0,...,0,0.0,13011078.0,0.0,0.0,0.0,0.0,0.0,2.0000,NO
282765,66946,36907,NaN,2019-05-24,569154.0,46197.0,422701.0,100256,569154,100.0,...,0,0.0,21630803.0,0.0,0.0,0.0,0.0,0.0,1.9500,NO


Se observan los tipos de datos y cantidad de valores nulos.

In [3]:
set_datos.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282767 entries, 0 to 282766
Data columns (total 35 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id_credito                         282767 non-null  int64  
 1   numero_radicacion                  282767 non-null  int64  
 2   numero_radicacion_credito_antiguo  11985 non-null   float64
 3   fecha_originacion                  282767 non-null  object 
 4   valor_esperado                     282767 non-null  float64
 5   capital                            282125 non-null  float64
 6   interes                            282125 non-null  float64
 7   seguros                            282767 non-null  int64  
 8   valor_recibido                     282767 non-null  int64  
 9   porcentaje_incorporacion           282767 non-null  float64
 10  causal_incorporacion               23450 non-null   object 
 11  tipo_causal                        2283

Se observa que nuestro dataset original consiste básicamente en especificaciones, cuota a cuota de distintos créditos.

## Tratamiento de datos.

De forma poreventiva, creamos un nuevo dataframe con los cambios que iremos haciendo.

In [4]:
cuotas = set_datos;

Notamos que hay varias columnas que parecen tener siempre los mismo valores. eliminan las columnas en que esto suceda.

In [5]:
def unique_cols(df):
    a = df.to_numpy()
    return (a[0] == a).all(0)
es_unico = unique_cols(set_datos)
cuotas = cuotas.loc[:,~es_unico]
cuotas.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282767 entries, 0 to 282766
Data columns (total 34 columns):
 #   Column                             Non-Null Count   Dtype  
---  ------                             --------------   -----  
 0   id_credito                         282767 non-null  int64  
 1   numero_radicacion                  282767 non-null  int64  
 2   numero_radicacion_credito_antiguo  11985 non-null   float64
 3   fecha_originacion                  282767 non-null  object 
 4   valor_esperado                     282767 non-null  float64
 5   capital                            282125 non-null  float64
 6   interes                            282125 non-null  float64
 7   seguros                            282767 non-null  int64  
 8   valor_recibido                     282767 non-null  int64  
 9   porcentaje_incorporacion           282767 non-null  float64
 10  causal_incorporacion               23450 non-null   object 
 11  tipo_causal                        2283

Ahora se eliminan las columnas que sean en su mayoria valores nulos

In [6]:
def null_cols(df,th):
    a = df.isna().sum()
    a = a / len(df) >= th
    return a.values
cuotas = cuotas.loc[:,~null_cols(cuotas,.5)]
cuotas.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 282767 entries, 0 to 282766
Data columns (total 28 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   id_credito                  282767 non-null  int64  
 1   numero_radicacion           282767 non-null  int64  
 2   fecha_originacion           282767 non-null  object 
 3   valor_esperado              282767 non-null  float64
 4   capital                     282125 non-null  float64
 5   interes                     282125 non-null  float64
 6   seguros                     282767 non-null  int64  
 7   valor_recibido              282767 non-null  int64  
 8   porcentaje_incorporacion    282767 non-null  float64
 9   periodo                     282767 non-null  object 
 10  estado_incorporacion        282750 non-null  object 
 11  estado                      282767 non-null  object 
 12  valor_otros_conceptos       282767 non-null  float64
 13  valor_amortiza

Finalmente, se borran las filas que contengan nulos

In [7]:
cuotas = cuotas.dropna()
cuotas

,id_credito,numero_radicacion,fecha_originacion,valor_esperado,capital,interes,seguros,valor_recibido,porcentaje_incorporacion,periodo,...,fecha_fin_plan_de_pagos,altura_mora_calculada,saldo_vencido,saldo_capital,capital_mora,intereses_corrientes_mora,seguros_mora,intereses_mora,gastos_cobranza,tasa_mensual_credito
3,1693,1,2013-08-15,124094.0,59740.0,60934.0,3420,124094,100.0,2015-12-30,...,2018-08-30,0,0.0,2769804.0,0.0,0.0,0.0,0.0,0.0,2.1535
4,1693,1,2013-08-15,124094.0,61027.0,59647.0,3420,124094,100.0,2016-01-30,...,2018-08-30,0,0.0,2708777.0,0.0,0.0,0.0,0.0,0.0,2.1535
5,1693,1,2013-08-15,124094.0,62341.0,58333.0,3420,124094,100.0,2016-02-28,...,2018-08-30,0,0.0,2646436.0,0.0,0.0,0.0,0.0,0.0,2.1535
6,1693,1,2013-08-15,124094.0,63684.0,56990.0,3420,124094,100.0,2016-03-30,...,2018-08-30,0,0.0,2582752.0,0.0,0.0,0.0,0.0,0.0,2.1535
7,1693,1,2013-08-15,124094.0,65055.0,55619.0,3420,124094,100.0,2016-04-30,...,2018-08-30,0,0.0,2517697.0,0.0,0.0,0.0,0.0,0.0,2.1535
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
282762,66562,36684,2019-05-07,364347.0,31259.0,267881.0,65207,364347,100.0,2019-05-30,...,2029-04-30,0,0.0,14067741.0,0.0,0.0,0.0,0.0,0.0,1.9000
282763,66563,36729,2019-05-20,182174.0,11971.0,139956.0,30247,182174,100.0,2019-05-30,...,2029-04-30,0,0.0,6528029.0,0.0,0.0,0.0,0.0,0.0,2.1400
282764,66694,36826,2019-05-16,347757.0,26703.0,260755.0,60299,347757,100.0,2019-05-30,...,2029-04-30,0,0.0,13011078.0,0.0,0.0,0.0,0.0,0.0,2.0000
282765,66946,36907,2019-05-24,569154.0,46197.0,422701.0,100256,569154,100.0,2019-05-30,...,2029-04-30,0,0.0,21630803.0,0.0,0.0,0.0,0.0,0.0,1.9500


## Obtención de variables

Observamos que "cuotas" no cuantifica comportamientos por crédito per se y aún menos alguna variable objetivo. Por tanto se crearán nuevos data frames que contengan esta información.